<a href="https://colab.research.google.com/github/jeanbouteiller-ds/tennis_prediction/blob/main/functions/github_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import subprocess
import sys

In [3]:
def install_package(package_name):
    try:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"Successfully installed {package_name}")
    except subprocess.CalledProcessError:
        print(f"Failed to install {package_name}") # Install the PyGitHub library

install_package("requests")
install_package("PyGitHub")
install_package("pandas")

In [4]:
import requests
import random
from bs4 import BeautifulSoup

from datetime import datetime, timedelta

import os
from github import Github
from io import StringIO

import json



In [28]:
# Replace these with your GitHub credentials
username = "jeanbouteiller-ds"

github_url = "https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/password.txt"
response = requests.get(github_url)

password= response.text.replace('\n','')
repository_name = "tennis_prediction"
branch_name = "main"  # Replace with your branch name


# Initialize the GitHub client
g = Github(username, password)

# Access the desired repository
repo = g.get_user().get_repo(repository_name)


def add_file_github(file_name,file_content):
  # Convert the list to a JSON string if it's a list
  if isinstance(file_content, list):
      # file_content = json.dumps(file_content)
      file_content = "\n".join(json.dumps(item) for item in file_content)

  try:
      file = repo.get_contents(file_name, ref=branch_name)
      sha = file.sha
  except Exception as e:
      sha = None
  if sha:
      repo.update_file(file_name, "Update file", file_content, sha, branch_name)
      # print(f"Updated {file_name} in {repository_name}")
  else:
      repo.create_file(file_name, "Create file", file_content, branch_name)
      # print(f"Created {file_name} in {repository_name}")

def download_file_from_github(file_url):
    response = requests.get(file_url)
    return response.content


def get_file_content(file_name):
  file_content=download_file_from_github("https://raw.githubusercontent.com/jeanbouteiller-ds/tennis_prediction/main/"+file_name)
  if file_name.split('.')[1]=='txt'  :
    file_content=str(file_content).strip()
    return(file_content)
  if file_name.split('.')[1]=='csv':
    file_content=pd.read_csv(io.BytesIO(file_content))
    return(file_content)
  if file_name.endswith('.json'):
        # If the file is a JSON file, parse its content and return a list
    try:
      file_content_lines = file_content.decode('utf-8').splitlines()  # Split lines
      file_content_list = [json.loads(line) for line in file_content_lines]
      return file_content_list
    except json.JSONDecodeError as e:
      print(f"Error parsing JSON file {file_name}: {e}")
      return None


In [29]:
get_file_content("update_all_matchs_report.json")

[{'te': 3},
 {'date': '2023-10-25 21:42',
  'Nb Duplicates': 9,
  'Nb New lines': 0,
  'Nb Total lines': 133417,
  'Nb New tournament_name': 0,
  'Nb Total tournament_name': 268}]